In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options


In [7]:
# view는 스크롤을 계속 내리는 코드를 작성해줘야 됨
# 스크롤을 끝까지 내린 후 링크를 모두 수집
# 수집한 링크를 하나씩 들어가서 데이터 수집
# 월별로 최대 1000개 크롤링 가능


In [8]:
# 크롤링 시작과 끝 날짜(달 별로 크롤링)
first_days = pd.date_range('2022/01/01','2022/11/17', freq='MS')   # 각 달의 첫 날
last_days = pd.date_range('2022/01/01','2022/11/17', freq='M')  # 각 달의 마지막 날

# keyword_list = ['인스타술집','이색술집','감성술집', '담금술집', '분위기술집', '정글포차', '인테리어술집', '새로운술집', '감성주점', '컨셉술집'] # 검색키워드 추가
# 지역 선정 방법 어떻게??
# keyword를 한줄로 검색해서 크롤링

# keyword_main = ['인스타술집','이색술집','감성술집', '컨셉술집']
# keyword_sub = ['을지로', '신당']
# keyword_list = []
# for i in keyword_main:
#     for j in keyword_sub:
#         keyword_list.append(j+i)
# print(keyword_list)
keyword_list = ['인스타술집','이색술집','감성술집', '컨셉술집','분위기술집']

In [2]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')#overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드

chrome_path = 'chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치


In [3]:
# 블로그 주소 수집
blog_url_list = []

# 네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)

    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)

    driver.get(url)
    time.sleep(3)

    url_want = 990  # 월별 1000개까지만 가능하기 때문에 개수 설정

    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭

    # 날짜 지정
    for k in range(len(first_days)):
        try:
            # 시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()

            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            scroll = (url_want/30)-1    # 한 페이지에 기본으로 30개 글이 나옴
            
            # 스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')  # 0부터 끝까지 스크롤 내림
                time.sleep(random.uniform(1,1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                if(new_height == last_height):
                    break

                last_height = new_height

            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a', class_ = 'api_txt_lines total_tit')   # 첫번째 view 제목 class

            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) # 주소 수집
            
            print(f'{keyword} 키워드 {first_days.year[k]}년 {first_days.month[k]}월', len(blog_url), '개 블로그 url 주소 수집') 

            driver.execute_script('window.scrollTo(0,0)')   # 스크롤 밑으로 내려가지 않게 해줌

        except:
            pass




NameError: name 'keyword_list' is not defined

In [11]:
len(blog_url_list)

13535

### 강사님 코드

In [12]:
# # 블로그 주소 수집

# blog_url_list = []

# #네이버에 키워드 검색
# for keyword in keyword_list:
#     url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
        
#     driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
#     driver.implicitly_wait(3)
    
#     driver.get(url)
#     time.sleep(3)
    
#     url_want = 990 #
    
#     driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭

#     #날짜 지정
#     for k in range(len(first_days)):
#         try:
#             #시작과 끝의 년월일 설정
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
#             driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()            
            
            
#             # 스크롤 조작
#             last_height = driver.execute_script('return document.body.scrollHeight')
            
#             scroll = (url_want/30)-1 #한 페이지에 기본으로 30개 글이 나옴
            
#             # 스크롤 다운
#             for i in range(int(scroll)):
#                 driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
#                 time.sleep(random.uniform(1,1.7))
                
#                 new_height = driver.execute_script('return document.body.scrollHeight')
                
#                 if new_height == last_height:
#                     break
                    
#                 last_height = new_height
                
                
#             soup = bs(driver.page_source, 'lxml')
#             blog_url = soup.find_all('a',class_='api_txt_lines total_tit')
            
#             # url 주소 수집
#             for i in blog_url:
#                 blog_url_list.append(i['href']) #주소 수집
                
#             print('{0} 키워드 {1}년{2}월'.format(keyword, 
#                                             first_days.year[k], frist_days.month[k]), 
#                   len(blog_url),'개 블로그 url 주소 수집')
            
            
#             driver.execute_script('window.scrollTo(0,0)')
            
#         except:
#             pass
            

In [13]:
# 수집 후 url list 중복 확인
real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

13190

In [14]:
# 파일 저장(블로그 주소)
k = pd.DataFrame(blog_url_list)
k.to_csv('url_list_술집.csv', encoding='utf-8-sig')

In [15]:
# 블로그 크롤링
url = pd.read_csv('url_list_술집.csv')
blog_url_list = list(url['0'])


In [16]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%200 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\user\AppData\Local\Temp\ipykernel_20276\2760139126.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


네이버 블로그 아닌 url:  https://wqnpiue9620.tistory.com/490
200개 블로그 크롤링 완료
400개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://allthatlookmarket.tistory.com/194
600개 블로그 크롤링 완료
800개 블로그 크롤링 완료
1000개 블로그 크롤링 완료
1200개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://sign.tistory.com/entry/%EC%84%9C%EC%9A%B8-%EC%84%9C%EB%8C%80%EB%AC%B8-%EC%B9%B4%ED%8E%98%EA%B0%9C%EC%97%85-%EC%84%A0%EB%AC%BC-%EA%B0%9C%EC%97%85%ED%99%94%ED%99%98-%EA%B5%AC%EB%A7%A4-%EC%88%9C%EC%9C%84
네이버 블로그 아닌 url:  https://ahfkdh.tistory.com/643
네이버 블로그 아닌 url:  https://gyayoon.tistory.com/entry/%EB%A7%88%EA%B3%A1-%EA%B0%80%EC%84%B1%EB%B9%84-%EC%8B%A0%EC%83%81%EC%88%A0%EC%A7%91-%EC%9D%B8%EC%8C%A9%EB%A7%A5%EC%A3%BC2%EC%B0%A8-%EC%88%A0%EC%A7%91-%EC%B6%94%EC%B2%9C
1400개 블로그 크롤링 완료
1600개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://korea-hotplace.tistory.com/4602
네이버 블로그 아닌 url:  https://madamamy.tistory.com/149
1800개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://korea-hotplace.tistory.com/4300
2000개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://eunbong.tistory.com/136
네이버 블로그 아닌 url:

In [17]:
len(time_list)

13126

In [18]:
data = {'time':time_list, 'review':review_list, 'comment':comment_list, 'like':like_list, 'url':url_list}
df = pd.DataFrame(data)
print('크롤링 소요 시간: ', time.time()-start)
print('총 {}개 블로그 크롤링 완료'.format(count))

크롤링 소요 시간:  59853.063157081604
총 13130개 블로그 크롤링 완료


In [19]:
# 파일 저장(블로그 주소)
d = pd.DataFrame(df)
d.to_csv('네이버 뷰 술집.csv', encoding='utf-8-sig')

In [20]:
df

,time,review,comment,like,url
0,2022. 7. 11. 17:27,CAFE 동대문 칵테일 바 / 신선한 냉장고문을 열고 들어가는...,[],0,https://blog.naver.com/belleocean77/222807566714
1,2022. 5. 27. 0:24,일상 진해 송학커피 창원핫플 정원카페 페파마미 ...,[],12,https://blog.naver.com/celine24005/222748141510
2,2022. 4. 29. 8:32,"ㄴ카페랑 빵 글루미커피/대구카페,일광욕,루프트탑,반월당...",[],4,https://blog.naver.com/ice7623/222715076158
3,2022. 10. 27. 21:09,수원맛집 [수원역] 하식당 | 분위기 좋은 감성술집 추천 ...,[],11,https://blog.naver.com/khj_1212/222912513272
4,2022. 4. 24. 10:06,♥ Yummy - 창원外 •울산 성남동• 더 바베큐 캠프 ...,[],26,https://blog.naver.com/zy4434/222709706549
...,...,...,...,...,...
13121,2022. 10. 5. 17:53,"1 대흥동 분위기좋은술집, 대전정든밤 흐림 ・ ...",[],36,https://blog.naver.com/julie9582/222892590298
13122,2022. 9. 7. 21:34,바삭한 맛집 대전 궁동 :: 신상 술집 빠삭 인스타감성 제대로...,[],7,https://blog.naver.com/othalso/222869601654
13123,2022. 9. 28. 9:10,restaurant 부산 동래역 분위기 술집 :: 소이만 ...,[],6,https://blog.naver.com/dyome_/222881975282
13124,2022. 10. 29. 22:54,맛집 부산 와인바 <소이만> 분위기좋은 동래 술집 ...,[],5,https://blog.naver.com/sycg260/222914319764
